In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
model_name = 'taide/Llama3-TAIDE-LX-8B-Chat-Alpha1'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir='/HDD/model_cache/'
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side='left',
    cache_dir='/HDD/model_cache/'
)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import json
from pathlib import Path
from datasets import Dataset

example_input = json.loads(Path('./labeled_datasets/example.json').read_text())['input']
example_output = json.loads(Path('./labeled_datasets/example.json').read_text())['output']
#27712
train_data = json.loads(Path('./dataset/combined_data_unique.json').read_text())
train_dataset = Dataset.from_list(train_data)
train_dataset

Dataset({
    features: ['text', 'category'],
    num_rows: 10
})

In [11]:

instruction = f'''
你是一個貼文審查專家，你的任務是判斷給定的貼文是否為詐騙，
你只能根據輸入內容生成以下格式的單個 JSON：
{{
    "text": "<貼文內容>",
    "label": "<標記結果 (0 或 1)>",
    "explination": "<標記理由>"
}}
除了上述 JSON 結果，請勿生成任何其他內容。
輸入輸出參考以下範例:
輸入: {example_input[0]}
輸出: {example_output[0]}

輸入: {example_input[1]}
輸出: {example_output[1]}

輸入: {example_input[2]}
輸出: {example_output[2]}

輸入: {example_input[3]}
輸出: {example_output[3]}

輸入: {example_input[4]}
輸出: {example_output[4]}

輸入: {example_input[5]}
輸出: {example_output[5]}

輸入: {example_input[6]}
輸出: {example_output[6]}

輸入: {example_input[7]}
輸出: {example_output[7]}

'''
sys = f'''{instruction}'''
print(sys)


你是一個貼文審查專家，你的任務是判斷給定的貼文是否為詐騙，
你只能根據輸入內容生成以下格式的單個 JSON：
{
    "text": "<貼文內容>",
    "label": "<標記結果 (0 或 1)>",
    "explination": "<標記理由>"
}
除了上述 JSON 結果，請勿生成任何其他內容。
輸入輸出參考以下範例:
輸入: {'text': '我們這邊有缺呦womanraisinghandlightskintone在家工作輕鬆賺飽飽'}
輸出: {'label': 1, 'explination': '這則貼文利用吸引人的語句，承諾在家輕鬆賺錢，未提供具體職業資訊，常見於詐騙訊息以吸引不知情的受害者。'}

輸入: {'text': 'Japaneseacceptablebutton不限平台私帶事由百家樂私帶群帶連倒五天噴了兩萬本金5000獲利32000內文放留言'}
輸出: {'label': 1, 'explination': '貼文內容提及私帶百家樂群組並宣稱高額獲利，這是常見的詐騙模式，誘導用戶加入非法賭博或詐騙群組，沒有可信度。'}

輸入: {'text': '人很奇怪有時候很想找人聊天有時候又想安安靜靜的沉浸在自己的世界'}
輸出: {'label': 0, 'explination': '這則貼文表達的是個人情感與日常心情分享，並無任何誘導性或詐騙特徵，屬於普通的社交貼文。'}

輸入: {'text': '能徵友文脆友單身來認識一下桃園新竹都能接受異地我83年次16053但希望有人追蹤我相互瞭解salutingface希望餘生有幸能多多指教要聊天哀居密我回覆喲facewithpeekingeye'}
輸出: {'label': 1, 'explination': '貼文以交友為名吸引目標，要求私訊追蹤，但內容缺乏真實性且帶有潛在誘導性，可能涉及交友詐騙或其他不當行為。'}

輸入: {'text': '台股今天繼續漲chartincreasing但是櫃買就沒這麼好BBU族群全面重挫是否拉回找買點船和飛機表現不錯但是AI軟體股更亮眼IC設計今天也是有點意思接著該怎麼做盤面好像一堆主流又好像沒有主流誰是真的誰是假的讓我們繼續看下去還好我的股都是紅的但一樣做了點調整股票投資'}
輸出: {'label': 0, 'e

In [6]:
generator = pipeline(
    model=model, 
    tokenizer=tokenizer,
    task="text-generation",
    device='cuda:1',
    batch_size=32
)

In [7]:
def prepare_prompt(datapoint):
    text = datapoint['text']
    query = f'輸入: {{"text": "{text}"}}'
    message = [
        {"role": "system", "content": sys},
        {"role": "user", "content": query},
    ]
    prompt = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        max_new_tokens=128,
        min_new_tokens=64
    )
    if prompt is None:
        raise ValueError(f"Prompt generation failed for text: {text}")
    datapoint['prompt'] = prompt
    return datapoint

In [12]:
from torch.utils.data import DataLoader
train_dataset = train_dataset.map(prepare_prompt, batched=False, num_proc=32)
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
train_dataset

num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.


Map (num_proc=10):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'category', 'prompt'],
    num_rows: 10
})

In [13]:
from tqdm import tqdm
result_dataset = {
    'text': [],
    'output': [],
}

for batch in tqdm(dataloader):
    prompts = batch['prompt'] 
    texts = batch['text']   
    
    outputs = generator(
        prompts, 
        max_new_tokens=64, 
        min_new_tokens=32,
        stop_strings="}",
        tokenizer = tokenizer,
        return_full_text=False,
    )
    
    result_dataset['text'].extend(texts)
    result_dataset['output'].extend(outputs)

100%|██████████| 1/1 [00:16<00:00, 16.55s/it]


In [14]:
len(result_dataset['output'])

10

In [15]:
import pandas as pd
pd.DataFrame(result_dataset).to_csv('dataset1.csv')

In [2]:
import pandas as pd
labeled_data = pd.read_csv('dataset.csv').drop(['Unnamed: 0'], axis=1)
labeled_data

,text,output
0,流量密碼,"[{'generated_text': ""輸出：{'label': 0, 'explinat..."
1,19歲知道男友手機密碼很有安全感29歲戶頭有200萬很有安全感,"[{'generated_text': '輸出：\n{\n""label"": 1,\n""exp..."
2,好像是欸昌董的車是流量密碼thumbsup,[{'generated_text': '我這邊就是把輸入的JSON格式的字典，對輸入的「t...
3,例如什麼讓我側耳傾聽redheart流量密碼在哪裡,"[{'generated_text': ""輸出：{'label': 1, 'explinat..."
4,查崗手機拿來洛燁漫不經心地將手機放在他手心上密碼我生日算了我挺相信你的說罷便將手機還給了他你...,"[{'generated_text': ""輸出：\n{'label': 0, 'explin..."
...,...,...
15842,好好工作賺錢吧,[{'generated_text': '這個輸入沒有形成一個有效的問題，因此無法根據該輸入...
15843,國一大安高工or長庚護專回頭看覺得國一志願設這兩個超不適合自己國二大安高工師大附中許願讀的高...,"[{'generated_text': '輸出：\n{\n""label"": 0,\n""exp..."
15844,中粉都不用賺錢不用上班嗎,"[{'generated_text': ""輸出: {'label': 1, 'explina..."
15845,原來可是我也疑惑嫌犯偷到手機怎麼沒登入哈哈,"[{'generated_text': ""輸出：{'label': 0, 'explinat..."


In [3]:
import re
def get_label(output):
    label_pattern = r"(label.*(1|0).*,)"
    match = re.search(label_pattern, output)
    return int(match.group(2))
def get_explination(output):
    label_pattern = r"(explination.*:([^}\"]*))"
    match = re.search(label_pattern, output)
    return match.group(2).strip().replace("'",'')

In [4]:
import json
import re
success_extract = []
failed_extract = []
for text, output in zip(labeled_data['text'], labeled_data['output']):
    try:
        label = get_label(output)
        explination = get_explination(output)
        success_extract.append({
            "text": text,
            "label": label,
            "explination": explination
        })
    except:
        failed_extract.append({
            "text": text,
            "output": output
        })

In [92]:
# from pathlib import Path
# output_path = Path('./dataset/training.json')
# with output_path.open('w', encoding='utf-8') as f:
#     json.dump(success_extract, f, ensure_ascii=False, indent=4)